# NEXUS tool: case study for the NWSAS basin - least-cost options calculations
In this notebook a case study for the NWSAS basin is covered using the `nexus_tool` package. The energy requirements for agricultural irrigation were previously calculated (check the water demand and energy demand case studies). In this case study, the least-cost options to supply the energy needs for agricultural irrigation are identified.

First import the package by running the following block:

In [156]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
%autoreload
import nexus_tool

In [158]:
file_path = r'nwsas_results/pumping_energy.csv'
df = nexus_tool.read_csv(file_path)

In [159]:
file_path = r'nwsas_data/Fuel_prices.xlsx'
fuel_price = nexus_tool.read_excel(file_path)

In [160]:
nwsas = nexus_tool.Model(df)

In [161]:
nwsas.df = nwsas.df.merge(fuel_price, on='country')

In [162]:
nwsas.create_wind_turbine('Wind power', life=20,
                          om_cost=0.02, capital_cost=1300,
                          efficiency=0.6)

In [163]:
nwsas.create_pv_system('Solar PV', life=15,
                       om_cost=0.01, capital_cost=1140,
                       efficiency=0.7)

In [164]:
nwsas.create_diesel_genset('Diesel set', life=10, om_cost=0.1,
                           capital_cost=938, fuel_cost=nwsas.df['Dprice1'],
                           fuel_req=0.095, efficiency=0.27, cf = 0.5,
                           emission_factor=2.7,env_cost=0)

In [165]:
nwsas.create_diesel_genset('Grid pump', life=10, om_cost=0.1,
                           capital_cost=845, fuel_cost=nwsas.df['Dprice1'],
                           fuel_req=1, efficiency=0.55, cf = 0.8,
                           emission_factor=0.728,env_cost=0)

In [167]:
nwsas.get_cf(['Wind power', 'Solar PV'])
nwsas.get_installed_capacity(['Wind power', 'Solar PV', 'Diesel set', 'Grid pump'])
nwsas.get_max_capacity(['Wind power', 'Solar PV', 'Diesel set', 'Grid pump'])
nwsas.end_year = 2030
nwsas.start_year = 2016
nwsas.discount_rate = 0.05
nwsas.get_lcoe(['Wind power', 'Solar PV', 'Diesel set', 'Grid pump'])

In [168]:
nwsas.df['wind_lcoe'] = nwsas.technologies['Wind power'].df['lcoe']
nwsas.df['pv_lcoe'] = nwsas.technologies['Solar PV'].df['lcoe']
nwsas.df['diesel_lcoe'] = nwsas.technologies['Diesel set'].df['lcoe']
nwsas.df['grid_lcoe'] = nwsas.technologies['Grid pump'].df['lcoe']
nwsas.df[['province','wind_lcoe','pv_lcoe', 'diesel_lcoe', 'grid_lcoe']].groupby('province').mean()

,wind_lcoe,pv_lcoe,diesel_lcoe,grid_lcoe
province,,,,
Adrar,10.359339,0.770447,0.552987,0.311350
Biskra,22.287251,0.950498,0.505705,0.284729
Djelfa,12.942626,0.897963,0.508724,0.286428
El Oued,18.697837,0.910149,0.541222,0.304726
Gabes,15.892456,0.940353,0.547916,0.308495
Ghadamis,11.332625,0.845636,0.520903,0.293286
Ghardaia,8.050849,0.814292,0.533570,0.300418
Gharyan,20.290699,1.140647,0.906686,0.510494
Illizi,10.186315,0.759492,0.537884,0.302847
